## [ Excel => DataFrame 저장 다루기 ]   

(1) 모듈 로딩 <hr>

In [84]:
import pandas as pd

(2) 데이터 준비 <hr>

In [85]:
file1 = '../DATA/Single Worksheet.xlsx'

In [86]:
file2 = '../DATA/Multiple Worksheets.xlsx'

(3) EXCEL => DataFrame 저장하기 <hr>

In [87]:
pd.read_excel(file1, header = None, usecols=[1, 3])
#pd.read_excel(file1, header = None, usecols=[1, 3], sheet_name =1)
#pd.read_excel(file1, header = None, usecols=[1, 3], index_col=1)

,1,3
0,Last Name,Gender
1,James,M
2,Hawkins,M
3,Day,F
4,Ruiz,F
5,Gomez,F


In [88]:
#아래 두 줄은 같다.
pd.read_excel(file2, sheet_name = 1) #sheet_name은 0부터 시작
pd.read_excel(file2, sheet_name = 'Data 3')

,First Name,Last Name,City,Gender
0,Robert,Miller,Seattle,M
1,Tara,Garcia,Phoenix,F
2,Raphael,Rodriguez,Orlando,M


## [ JSON ==> DataFrame 저장하기 ]

(1) 데이터 준비 <hr>

In [89]:
file3 = '../DATA/nobel.json'

(2) JSON => DataFrame으로 저장

In [90]:
nobelDF = pd.read_json(file3)
nobelDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prizes  646 non-null    object
dtypes: object(1)
memory usage: 5.2+ KB


(3) 데이터 확인

In [91]:
print(nobelDF.head(3), nobelDF.tail(3), sep ='\n\n')

                                              prizes
0  {'year': '2019', 'category': 'chemistry', 'lau...
1  {'year': '2019', 'category': 'economics', 'lau...
2  {'year': '2019', 'category': 'literature', 'la...

                                                prizes
643  {'year': '1901', 'category': 'peace', 'laureat...
644  {'year': '1901', 'category': 'physics', 'laure...
645  {'year': '1901', 'category': 'medicine', 'laur...


(4) 데이터 추출

In [92]:
# 행 추출 => iloc[] / loc[]
type(nobelDF.iloc[0]), nobelDF.iloc[0]

(pandas.core.series.Series,
 prizes    {'year': '2019', 'category': 'chemistry', 'lau...
 Name: 0, dtype: object)

In [93]:
type(nobelDF.iloc[0]['prizes'])

dict

In [94]:
oneData = nobelDF.iloc[0]['prizes']
type(oneData)

dict

In [95]:
oneData.keys(), oneData.items()

(dict_keys(['year', 'category', 'laureates']),
 dict_items([('year', '2019'), ('category', 'chemistry'), ('laureates', [{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}])]))

In [96]:
for k, v in oneData.items() :
    print(f'[{k}]\n===>{v}')
    if k == 'laureates' : 
        for d in v :
            for k1, v1 in d in d.items() :
                print('-', k1, v1)

[year]
===>2019
[category]
===>chemistry
[laureates]
===>[{'id': '976', 'firstname': 'John', 'surname': 'Goodenough', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '977', 'firstname': 'M. Stanley', 'surname': 'Whittingham', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}, {'id': '978', 'firstname': 'Akira', 'surname': 'Yoshino', 'motivation': '"for the development of lithium-ion batteries"', 'share': '3'}]


TypeError: 'bool' object is not iterable

In [97]:
# 행 추출 => iloc[[n1....nn]] / loc[]
nobelDF.iloc[[0, 3]]

,prizes
0,"{'year': '2019', 'category': 'chemistry', 'lau..."
3,"{'year': '2019', 'category': 'peace', 'laureat..."


- json 중첩 데이터의 다중 반복문 ==> 단일 리스트 형태 처리 : 평탄화 / 정규화

In [98]:
pd.json_normalize(oneData)

,year,category,laureates
0,2019,chemistry,"[{'id': '976', 'firstname': 'John', 'surname':..."


In [99]:
pd.json_normalize(oneData, record_path='laureates')

,id,firstname,surname,motivation,share
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3


In [100]:
pd.json_normalize(oneData, record_path='laureates', meta = ['year', 'category'])

,id,firstname,surname,motivation,share,year,category
0,976,John,Goodenough,"""for the development of lithium-ion batteries""",3,2019,chemistry
1,977,M. Stanley,Whittingham,"""for the development of lithium-ion batteries""",3,2019,chemistry
2,978,Akira,Yoshino,"""for the development of lithium-ion batteries""",3,2019,chemistry


- 모든 행에 대한 정규화/평탄화

In [101]:
#컬럼 읽기 => DF객체변수명['컬럼명'], DF객체변수명.컬럼명
#==> 모든 행에 record_path에 해당하는 키(key)가 존재해야함. 
#==> 없는 행에 record_path에 해당하는 키 추가
#pd.json_normalize(nobelDF.prizes, record_path = "laureates") # "laureates"가 없는 요소가 있어서 keyError 발생.

In [102]:
#컬럼 읽기 => DF객체변수명['컬럼명'], DF객체변수명.컬럼명
#==> 모든 행에 record_path에 해당하는 키(key)가 존재해야함. 
#==> 없는 행에 record_path에 해당하는 키 추가
for data in nobelDF.prizes :
    print(type(data), data.keys(), end = '\n\n')
    #해당 dict에 laureates 키가 없다면 추가.
    data.setdefault('laureates', []) #해당 키가 있으면 두고 없으면 추가.

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'overallMotivation', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'overallMotivation', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category', 'laureates'])

<class 'dict'> dict_keys(['year', 'category

In [103]:
nobelDF2 = pd.json_normalize(nobelDF.prizes, record_path='laureates', meta = ['year', 'category'])

In [123]:
nobelDF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          950 non-null    object
 1   firstname   950 non-null    object
 2   surname     921 non-null    object
 3   motivation  950 non-null    object
 4   share       950 non-null    object
 5   year        950 non-null    object
 6   category    950 non-null    object
dtypes: object(7)
memory usage: 52.1+ KB


(7) JSON의 다양한 방식의 DataFrame 저장하기 => orient

In [105]:
json_data='''
[
{"id" : "1","name":"sravan","age":22},
{"id" : "2","name":"harsha","age":22},
{"id" : "3","name":"deepika","age":21},
{"id" : "4","name":"jyothika","age":23}
]
''' #json에서 []는 리스트 X, <array>

- oriend = 'record' 설정 => 행단위로 JSON 파일 데이터를 행 단위로 읽어서 DataFrame에 저장.

In [106]:
df = pd.read_json(json_data, orient = 'record')
print(df)

   id      name  age
0   1    sravan   22
1   2    harsha   22
2   3   deepika   21
3   4  jyothika   23


In [116]:
# 데이터 => 키 : {값}
json_data2='''

{"student-1" : {"id" : "1","name":"sravan","age":22},
"student-2" : {"id" : "2","name":"harsha","age":22},
"student-3" : {"id" : "3","name":"deepika","age":21},
"student-4" : {"id" : "4","name":"jyothika","age":23}}

'''

In [117]:
# 키 => 인덱스로 설정
df = pd.read_json(json_data2, orient = 'index')
print(df)

           id      name  age
student-1   1    sravan   22
student-2   2    harsha   22
student-3   3   deepika   21
student-4   4  jyothika   23


In [113]:
# 키 => 컬럼으로 설정
df = pd.read_json(json_data2, orient = 'columns') #디폴트는 컬럼!
print(df)

                                  student-1  \
0  {'id': '1', 'name': 'sravan', 'age': 22}   
1                                       NaN   
2                                       NaN   
3                                       NaN   

                                  student-2  \
0                                       NaN   
1  {'id': '2', 'name': 'harsha', 'age': 22}   
2                                       NaN   
3                                       NaN   

                                   student-3  \
0                                        NaN   
1                                        NaN   
2  {'id': '3', 'name': 'deepika', 'age': 21}   
3                                        NaN   

                                    student-4  
0                                         NaN  
1                                         NaN  
2                                         NaN  
3  {'id': '4', 'name': 'jyothika', 'age': 23}  


In [119]:
# 키 => 컬럼으로 설정
df = pd.read_json(json_data2, orient = 'columns').T #열 <=> 행
print(df)

          id      name age
student-1  1    sravan  22
student-2  2    harsha  22
student-3  3   deepika  21
student-4  4  jyothika  23


In [120]:
# JSON Format 
json_data3 = '''
[
    [ "1", "sravan",22 ],
    [ "2", "harsha",22 ],
    [ "3", "deepika",21 ],
    [ "4", "jyothika",23 ]
]
'''#확장자가 json임에도 key 없이 값만 존재.

In [121]:
df = pd.read_json( json_data3, orient='values' ) # 값만 들어있음을 명시
print(df)

   0         1   2
0  1    sravan  22
1  2    harsha  22
2  3   deepika  21
3  4  jyothika  23
